In [1]:
!pip install gradio google-generativeai

In [23]:
# Imports

import os
import re
import zipfile
import gradio as gr
from dotenv import load_dotenv
import google.generativeai as genai
import json, shutil, tempfile

In [24]:
# Load API Key

load_dotenv()
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

genai.configure(api_key=GOOGLE_API_KEY)
model = genai.GenerativeModel("gemini-1.5-flash")

In [25]:
# Predefined templates

TEMPLATES = {
    "Portfolio": "Make a portfolio site with About, Projects, Contact sections.",
    "Blog": "Make a blog site with posts, categories, and comments.",
    "E-Commerce": "Make a store with product listing, cart, and checkout.",
    "Landing Page": "Make a single-page landing site with hero, features, and contact form."
}

In [26]:
def safe_json_loads(raw_text):
    """
    Try to clean raw LLM output and parse JSON safely.
    """
    if not raw_text or not raw_text.strip():
        raise ValueError("Empty response from model")

    # Remove code block markers like ```json ... ```
    cleaned = re.sub(r"```(json|JSON)?", "", raw_text)
    cleaned = re.sub(r"```", "", cleaned)

    # Strip leading/trailing whitespace
    cleaned = cleaned.strip()

    # Try parsing
    return json.loads(cleaned)

In [29]:
def generate_website(idea, template, color, font, layout):
    if template != "None":
        idea = TEMPLATES[template]

    design = f"""
    Design Preferences:
    - Color scheme: {color if color else 'default'}
    - Font: {font if font else 'default'}
    - Layout: {layout if layout else 'default'}
    """

    prompt = f"""
    You are a website generator. Based on the user's idea, create a fully functional website.
    
    Output in JSON format with these keys:
    - frontend: React + Tailwind code
    - backend: Flask + SQLite code
    - db_schema: SQLite schema.sql
    - readme: Project setup instructions

    User Idea: {idea}
    {design}
    """

    response = model.generate_content(prompt)
    raw_text = response.text

    try:
        data = safe_json_loads(raw_text)
    except Exception as e:
        return (
            f"❌ Failed to parse AI response: {e}",
            "", "", "", "", None,
            raw_text
        )

    # Save files into a temp project folder
    tmpdir = tempfile.mkdtemp()
    os.makedirs(f"{tmpdir}/frontend", exist_ok=True)
    os.makedirs(f"{tmpdir}/backend", exist_ok=True)
    os.makedirs(f"{tmpdir}/db", exist_ok=True)

    with open(f"{tmpdir}/frontend/App.jsx", "w", encoding="utf-8") as f:
        f.write(str(data.get("frontend", "")))

    with open(f"{tmpdir}/backend/server.py", "w", encoding="utf-8") as f:
        f.write(str(data.get("backend", "")))

    with open(f"{tmpdir}/db/schema.sql", "w", encoding="utf-8") as f:
        f.write(str(data.get("db_schema", "")))

    with open(f"{tmpdir}/README.md", "w", encoding="utf-8") as f:
        f.write(str(data.get("readme", "")))


    # Zip project
    zip_path = shutil.make_archive("website_project", "zip", tmpdir)

    return (
        "✅ Website generated successfully!",
        data.get("frontend", ""),
        data.get("backend", ""),
        data.get("db_schema", ""),
        data.get("readme", ""),
        zip_path,
        raw_text
    )

In [30]:
# Gradio UI
with gr.Blocks(title="Sitecraft") as demo:
    with gr.Row():
        with gr.Column():
            gr.HTML("<h1 style='margin: 0; font-size: 2.2em; color: #2563eb; font-weight: 800; text-align:center;'>⚡<b>SiteCraft</b></h1>")
            gr.HTML("<p style='font-size: 1.2em; color: #374151; margin-top: 4px; text-align:center;'>Crafting websites with AI magic</p>")
    
    with gr.Row():
        idea = gr.Textbox(label="Describe your website idea", placeholder="e.g., A portfolio site with 3 sections...")
        template = gr.Dropdown(["None"] + list(TEMPLATES.keys()), label="Or choose a template")

    with gr.Accordion("🎨 Design Preferences (optional)", open=False):
        color = gr.Textbox(label="Color Scheme", placeholder="e.g., Blue & White")
        font = gr.Textbox(label="Font", placeholder="e.g., Sans-serif, Roboto")
        layout = gr.Textbox(label="Layout", placeholder="e.g., One-page scroll, Dashboard")

    btn = gr.Button("Generate Website")

    status = gr.Label()
    with gr.Tab("Frontend (React + Tailwind)"):
        frontend_code = gr.Code(language="javascript")
    with gr.Tab("Backend (Flask + SQLite)"):
        backend_code = gr.Code(language="python")
    with gr.Tab("Database Schema"):
        db_code = gr.Code(language="sql")
    with gr.Tab("README"):
        readme_text = gr.Code(language="markdown")
    with gr.Tab("🔍 Debug (Raw LLM Response)"):
        debug_text = gr.Textbox(lines=15, interactive=False, label="Raw Response")

    file = gr.File(label="⬇️ Download Project", type="filepath")

    btn.click(
        fn=generate_website,
        inputs=[idea, template, color, font, layout],
        outputs=[status, frontend_code, backend_code, db_code, readme_text, file, debug_text]
    )

demo.launch()

* Running on local URL:  http://127.0.0.1:7867
* To create a public link, set `share=True` in `launch()`.
